In [1]:
import pandas as pd
import flair.datasets as datasets

from tqdm import tqdm
from test import get_candidates
from src.utils import getEntityInfo

In [2]:
aida = datasets.NEL_ENGLISH_AIDA()
DBPedia_desc = pd.read_csv('./data/DBPedia.csv')
DBPedia_entites = DBPedia_desc['title'].unique()

2021-11-16 12:04:59,048 Reading data from C:\Users\athar\.flair\datasets\nel_english_aida
2021-11-16 12:04:59,051 Train: C:\Users\athar\.flair\datasets\nel_english_aida\train
2021-11-16 12:04:59,054 Dev: C:\Users\athar\.flair\datasets\nel_english_aida\testa
2021-11-16 12:04:59,055 Test: C:\Users\athar\.flair\datasets\nel_english_aida\testb


In [3]:
# Get all valid candidate entities for the test set
entities = []

for i in range(1163, 1394):
	cands = get_candidates(i)
	entities += list(cands['url'].unique())

for i in aida.test:
	entities += [j.tag for j in i.get_spans()]

entities = [i.split('/')[-1] for i in set(entities)]
entities = list(set(entities))

In [4]:
def get_desc(entity):
	try:
		return DBPedia_desc[DBPedia_desc['title'] == entity]['description'].values[0]
	except Exception as e:
		return ''

In [5]:
entity_desc = []
missing = []
for i in tqdm(entities):
	desc = get_desc(i).replace('\n', ' ')
	if desc == '':
		missing.append(i)
	else:
		entity_desc.append([i, desc])

100%|██████████| 14944/14944 [1:29:30<00:00,  2.78it/s]


In [6]:
invalid = []
for i in tqdm(missing):
	info, desc = getEntityInfo(i, inc_extract=True)
	if info != 'Topics referred to by the same term' and desc != '':
		entity_desc.append([i, desc.replace('\n', ' ')])
	else:
		invalid.append(i)

100%|██████████| 1532/1532 [19:42<00:00,  1.30it/s]


In [7]:
entity_desc_df = pd.DataFrame(entity_desc, columns=['entity', 'description'])

In [8]:
entity_desc_df.to_csv('data/entity_desc.csv', index=False)